# Custom API Endpoints

The [txtai API](https://neuml.github.io/txtai/api/) is a web-based service backed by [FastAPI](https://fastapi.tiangolo.com/). Semantic search, LLM orchestration and Language Model Workflows can all run through the API.

While the API is extremely flexible and complex logic can be executed through YAML-driven workflows, some may prefer to create an endpoint in Python.

This notebook introduces API extensions and shows how they can be used to define custom Python endpoints that interact with txtai applications.

# Install dependencies

Install `txtai` and all dependencies.

In [1]:
%%capture
!pip install git+https://github.com/neuml/txtai#egg=txtai[api] datasets

# Define the extension

First, we'll create an application that defines a persistent embeddings database and LLM. Then we'll combine those two into a RAG endpoint through the API.

In [2]:
%%writefile app.yml

# Embeddings index
writable: true
embeddings:
  hybrid: true
  content: true

# LLM pipeline
llm:
  path: google/flan-t5-large
  torch_dtype: torch.bfloat16

Writing app.yml


The code below creates an API endpoint at `/rag`. This is a `GET` endpoint that takes a `text` parameter as input.

In [3]:
%%writefile rag.py
from fastapi import APIRouter
from txtai.api import application, Extension


class RAG(Extension):
    """
    API extension
    """

    def __call__(self, app):
        app.include_router(RAGRouter().router)


class RAGRouter:
    """
    API router
    """

    router = APIRouter()

    @staticmethod
    @router.get("/rag")
    def rag(text: str):
        """
        Runs a retrieval augmented generation (RAG) pipeline.

        Args:
            text: input text

        Returns:
            response
        """

        # Run embeddings search
        results = application.get().search(text, 3)
        context = " ".join([x["text"] for x in results])

        prompt = f"""
        Answer the following question using only the context below.

        Question: {text}
        Context: {context}
        """

        return {
            "response": application.get().pipeline("llm", (prompt,))
        }

Writing rag.py


# Start the API instance

Let's start the API with the RAG extension.

In [4]:
!CONFIG=app.yml EXTENSIONS=rag.RAG nohup uvicorn "txtai.api:app" &> api.log &
!sleep 60

# Create the embeddings database

Next, we'll create the embeddings database using the `ag_news` dataset. This is a set of news stories from the mid 2000s.

In [ ]:
from datasets import load_dataset
import requests

ds = load_dataset("ag_news", split="train")

# API endpoint
url = "http://localhost:8000"
headers = {"Content-Type": "application/json"}

# Add data
batch = []
for text in ds["text"]:
  batch.append({"text": text})
  if len(batch) == 4096:
    requests.post(f"{url}/add", headers=headers, json=batch, timeout=120)
    batch = []

if batch:
    requests.post(f"{url}/add", headers=headers, json=batch, timeout=120)

# Build index
index = requests.get(f"{url}/index")

# Run queries

Now that we have a knowledge source indexed, let's run a set of queries. The code below defines a method that calls the `/rag` endpoint and retrieves the response. Keep in mind this dataset is from 2004.

While the Python Requests library is used in this notebook, this is a simple web endpoint that can be called from any programming language.

In [14]:
def rag(text):
    return requests.get(f"{url}/rag?text={text}").json()["response"]

rag("Who is the current President?")

'George W. Bush'

In [15]:
rag("Who lost the presidential election?")

'John Kerry'

In [16]:
rag("Who won the World Series?")

'Boston'

In [17]:
rag("Who did the Red Sox beat to win the world series?")

'Cardinals'

In [18]:
rag("What major hurricane hit the USA?")

'Charley'

In [19]:
rag("What mobile phone manufacturer has the largest current marketshare?")

'Nokia'

# Wrapping up

This notebook showed how a txtai application can be extended with custom endpoints in Python. While applications have a robust workflow framework, it may be preferable to write complex logic in Python and this method enables that.